In [1]:
import numpy as np

import ngsolve 
from ngsolve.webgui import Draw
import netgen.geom2d

### Implicit Eulerian method for the heat equation 

WE are solving
$$\partial_t = \Delta u  $$ 
Weak form 
$$\int  \partial_t u v\ dx +  \int \nabla u \nabla v dx =0 \qquad \forall v$$

Implicit Euler
$$ \int \frac{(u^{n+1} - u^n)}{dt} v \ dx + \int \nabla u^{n+1} \nabla v dx  = 0 $$

Let $$ Mu = \int u v dx  \text { and  } 
L u = \int \nabla u \nabla v dx $$ 
\begin{align}
 M u^{n+1} - M u^n + dt L u^{n+1} &= 0 \\
M u^{n+1} + dt L u^{n+1} - M u^n  - dt L u^n + dt L u^n &= 0  \\
(M + dt L) ( u^{n+1} - u^n) + dt L u^n &= 0 
\end{align}
Let $$\delta = ( u^{n+1} - u^n) \text{  and } M_* = (M + dt L)$$
\begin{align}
\delta = M_*^{-1} (- dt L u^n)
\end{align}

In [2]:
# https://docu.ngsolve.org/latest/i-tutorials/unit-4.1.2-csg2d/csg2d.html
geo = netgen.geom2d.CSG2d()

# make a circle with a hole in it!
circle1 = netgen.geom2d.Circle( center=(0,0), radius=1.0,  bc="outer" )
circle2 = netgen.geom2d.Circle( center=(0.1,0), radius=0.3,  bc="inner" )

annulus = circle1 - circle2
geo.Add(annulus)

In [3]:
maxh_mesh=0.07
mesh = ngsolve.Mesh( geo.GenerateMesh(maxh=maxh_mesh))
Draw(mesh)
mesh.GetBoundaries()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

('outer', 'outer', 'outer', 'outer', 'inner', 'inner', 'inner', 'inner')

In [4]:
korder = 2
fes = ngsolve.H1(mesh, order=korder)    # if you want dirichlet BC:  ,dirichlet="inner|outer") 
u,v =  fes.TnT()  # create test and trial functions  

In [5]:
nu=0.1
dt = 0.01

# Laplacian  = L above
a = ngsolve.BilinearForm(fes)  # holds Laplacian operator for both fields 
a += nu*ngsolve.grad(u)*ngsolve.grad(v)*ngsolve.dx # This is laplacian 
a.Assemble()

# Mass matrix  = M above
m = ngsolve.BilinearForm(fes)  
m += u*v*ngsolve.dx 
m.Assemble()

# Inverse of mass matrix  M^-1
im = m.mat.CreateMatrix()
invm = im.Inverse(freedofs=fes.FreeDofs())  # in case we want an inverse!

# Mstar = dt L + M 
mstar = ngsolve.BilinearForm(fes) 
mstar +=  dt*nu*ngsolve.grad(u)*ngsolve.grad(v)*ngsolve.dx  + u*v*ngsolve.dx 
mstar.Assemble()

# Inverse of Mstar  = M_*^-1
ims = mstar.mat.CreateMatrix()
invmstar = ims.Inverse(freedofs=fes.FreeDofs())

# create thing to hold the solution 
gfu = ngsolve.GridFunction(fes)


In [6]:
# create initial conditions 
rho_0 = 1.0
x0=-0.5; y0=0.0; sig0=0.5; sig2= sig0*sig0
r2 = (ngsolve.x -x0) * (ngsolve.x  - x0) + (ngsolve.y-y0) * (ngsolve.y -y0)
rho_bump = 0.01*ngsolve.exp(-0.5*r2/sig2) + rho_0 # is a coefficient function 

# set the initial density field in the gfu solution field 
gfu.Set(rho_bump)
scene = Draw(gfu)

t=0 # initial time

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [7]:

tend=3.0  #end time 

while (t < tend-0.5*dt):
    res = -dt*a.mat* gfu.vec
    gfu.vec.data += invmstar * res
    scene.Redraw()
    t = t + dt;